In [1]:
import numpy as np
# import matplot for visualization
import matplotlib.pyplot as plt
#to see the plots inside of the notebook
%matplotlib inline
# import tensorflow and required layers
# note that tensorflow.contrib.layers was previously migrated from TF Slim.
import tensorflow as tf
from tensorflow.contrib.layers import conv2d, fully_connected, max_pool2d, repeat
from scipy import misc
slim = tf.contrib.slim


# create a session
session = tf.Session()


rois=[]
with open('data/regions.txt') as f:
    rois =[x.strip() for x in  f.readlines()]
    rois.pop(0)#no need for first line

numberOfInputImages = 1
width=600
height=600
numberOFChannels=3

roiOutputWidth=7
roiOutputHeight=7

inputs = tf.placeholder(dtype=tf.float32,shape=(numberOfInputImages,width,height,numberOFChannels))
#with tf.variable_scope('vgg_16', inputs, [1000]) as sc:
#        with slim.arg_scope([conv2d, fully_connected, max_pool2d]):
net = repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
net = max_pool2d(net, [2, 2], scope='pool1')
net = repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
net = max_pool2d(net, [2, 2], scope='pool2')
net = repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
net = max_pool2d(net, [2, 2], scope='pool3')
net = repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
net = max_pool2d(net, [2, 2], scope='pool4')
net = repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')


In [2]:
slices = []
for line in rois:
    #line=rois[2]
    l=line.split()
    label=l[0]
    x1=int(l[1])
    x2=int(l[3])
    y1= int(l[2])
    y2=int(l[4])
    #print(type(net.shape[1]))
    #print('shape:')
    #print((int(net.shape[1])))
    xBegin=int(x1*int(net.shape[1])/int(inputs.shape[1]))
    yBegin = int(y1*int(net.shape[1])/int(inputs.shape[1]))
    xEnd=int(x2*int(net.shape[1])/int(inputs.shape[1]))
    yEnd =int( y2*int(net.shape[1])/int(inputs.shape[1]))
    xDiff=(xEnd-xBegin)
    yDiff=(yEnd-yBegin)
    sl = tf.slice(
            net,
            [0,xBegin,yBegin,0],
            [int(net.shape[0]),xDiff,yDiff,int(net.shape[3])]
        )
    print('input')
    print(net.shape)
    print('kernel')
    roiKernel=[int(xDiff/roiOutputWidth), int(yDiff/roiOutputHeight)]
    print(roiKernel)
    sl = max_pool2d(sl, roiKernel,roiKernel, scope='ROIpool'+label)
    print('output from ROI')
    print(sl.shape)
    #net = max_pool2d(net, [2, 2], scope='pool5')
    # Use conv2d instead of fully_connected layers.
    sl = tf.slice(
            net,
            [0,0,0,0],
            [int(net.shape[0]),roiOutputWidth,roiOutputHeight,int(net.shape[3])]
        )
    print('after slicing')
    print(sl.shape)
    slices.append(sl)
net = tf.stack(slices,axis=1)
#net = conv2d(tf.stack(slices,axis=0), 4096, [7, 7], padding='VALID', scope='fc6')
#net = conv2d(net, 4096, [1, 1], scope='fc7')
#net = conv2d(net, 1000, [1, 1], activation_fn=None, normalizer_fn=None, scope='fc8')
print('output')
print(net.shape)

input
(1, 37, 37, 512)
kernel
[2, 4]
output from ROI
(1, 9, 8, 512)
after slicing
(1, 7, 7, 512)
input
(1, 37, 37, 512)
kernel
[1, 2]
output from ROI
(1, 7, 7, 512)
after slicing
(1, 7, 7, 512)
input
(1, 37, 37, 512)
kernel
[1, 1]
output from ROI
(1, 9, 11, 512)
after slicing
(1, 7, 7, 512)
output
(1, 3, 7, 7, 512)


In [3]:
init_op = tf.global_variables_initializer()
session.run(init_op)
img = misc.imresize(misc.imread('images/people.jpg'),size=[600,600]).reshape([1,600,600,3])
RoisOutput = session.run([net],feed_dict={inputs:img})
print(RoisOutput[0].shape)

#(1, 3, 7, 7, 512) , 1 image, 3 rois, 7x7, 512 channels

(1, 3, 7, 7, 512)
